# Grundlagen – Teil 1: Dispersion der Prisma

Beim Durchgang eines Lichtstrahls durch ein Prisma wird der Strahl an beiden Grenzflächen gebrochen.
Das Verhalten des Lichts wird durch das Brechungsgesetz von Snellius beschrieben:
$$n_1 \sin(\alpha) = n_2 \sin(\beta)$$

Hierbei gilt:
- $n_1, n_2$ - Brechungsindizes der beteiligten Medien (Luft → Glas → Luft)
- $\alpha$ - Einfallswinckel
- $\beta$ - Brechungswinkel

---

Für eine gleichseitige Prisma mit Scheitelwinkel  $\varepsilon$ (z.B. 60°) ergibt sich für die gesamte Ablenkung $\delta$ des Strahls:
$$\delta = \alpha_1 + \alpha_2 = \epsilon$$

Im allgemeinen Fall ist der Strahlengang asymmetrisch, d. h. die Einfalls- und Austrittswinkel unterscheiden sich.
Bei der minimalen Ablenkung $\delta_{min}$ verläuft der Strahl jedoch __symmetrisch__:

$$\alpha_1 = \alpha_2 \text{ und } \beta_1 = \beta_2 ={\varepsilon\over{2}}$$

Unter dieser Bedingung lässt sich der __Brechungsindex des Prismamaterials__ berechnen zu:
$$n = {\sin ({{\delta_{min} + \varepsilon} \over {2}}) \over \sin ({{\varepsilon} \over {2}})}$$

In [11]:

import math
import numpy as np
from typing import List, Tuple, Optional

# =========================
# HILFSFUNKTIONEN
# =========================
def safe_div(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    return a / np.maximum(b, eps)

def rmse(y: np.ndarray, yhat: np.ndarray) -> float:
    return float(np.sqrt(np.mean((y - yhat) ** 2)))

def compute_n_from_delta(delta_min_deg: np.ndarray, epsilon_deg: float) -> np.ndarray:
    """n = sin((δ_min + ε)/2) / sin(ε/2), Winkel in Grad."""
    delta = np.deg2rad(delta_min_deg)
    eps = math.radians(epsilon_deg)
    return np.sin((delta + eps) / 2.0) / np.sin(eps / 2.0)

def cauchy_fit(lambda_nm: np.ndarray, n: np.ndarray) -> Tuple[float, float, float, np.ndarray]:
    """
    Cauchy-Modell: n(λ) = A + B/λ^2 + C/λ^4, λ in μm.
    Linearer Least Squares über X = [1, 1/λ^2, 1/λ^4].
    """
    l_um = lambda_nm / 1000.0
    X = np.column_stack([np.ones_like(l_um), 1.0/(l_um**2), 1.0/(l_um**4)])
    coeffs, *_ = np.linalg.lstsq(X, n, rcond=None)
    n_fit = X @ coeffs
    A, B, C = map(float, coeffs)
    return A, B, C, n_fit

def find_peaks_simple(x_nm: np.ndarray, y: np.ndarray,
                      min_prom: float, min_dist_nm: float) -> List[int]:
    """
    Einfache Peak-Suche:
    - lokales Maximum: y[i] > y[i±1]
    - Prominenz ~ Differenz zu direkten Nachbarn
    - Mindestabstand in nm
    (Für präzisere Analysen wäre scipy.signal.find_peaks vorzuziehen.)
    """
    idxs: List[int] = []
    last_x = -1e9
    for i in range(1, len(y) - 1):
        if y[i] > y[i-1] and y[i] > y[i+1]:
            prom = y[i] - max(y[i-1], y[i+1])
            if prom >= min_prom and (x_nm[i] - last_x) >= min_dist_nm:
                idxs.append(i)
                last_x = x_nm[i]
    return idxs

def nearest_lines(query_nm: List[float], ref_nm: List[float], tol_nm: float = 2.5):
    """Zuordnung gemessener Peaks zu Referenzlinien (±tol_nm)."""
    out = []
    for q in query_nm:
        diffs = [abs(q - r) for r in ref_nm]
        j = int(np.argmin(diffs))
        d = diffs[j]
        out.append((q, ref_nm[j] if d <= tol_nm else None, d))
    return out

# =========================
# TEIL 1 — DISPERSION DER PRISMA
# =========================

# ---------- EINGABEDATEN (nur Arrays) ----------
EPSILON_DEG = 60.0  # Scheitelwinkel der Prisma (z. B. 60°)
delta_min_deg = [41.1, 41.0, 40.8, 40.7, 40.5]            # δ_min [Grad] (ersetzen!)
lambda_nm     = [579.07, 576.96, 546.07, 491.60, 435.83]  # λ [nm] passend zu δ_min

DO_CAUCHY_FIT = True  # Fit ein/aus

# ---------- Rechnung ----------
lam_nm = np.array(lambda_nm, dtype=float)
dmin_deg = np.array(delta_min_deg, dtype=float)
assert lam_nm.shape == dmin_deg.shape, "Fehler: λ und δ_min müssen gleich lang sein."

n_vals = compute_n_from_delta(dmin_deg, EPSILON_DEG)

print("\n=== TEIL 1 — DISPERSION DER PRISMA ===")

print("\nBerechnete n(λ):")
for l, dlt, n in zip(lam_nm, dmin_deg, n_vals):
    print(f"  λ = {l:7.2f} nm | δ_min = {dlt:5.2f}° | n = {n:.6f}")

if DO_CAUCHY_FIT and len(lam_nm) >= 3:
    A, B, C, n_fit = cauchy_fit(lam_nm, n_vals)
    resid = n_vals - n_fit
    print("\nCauchy-Fit (λ in μm):   n(λ) = A + B/λ^2 + C/λ^4")
    print(f"  A = {A:.6f}")
    print(f"  B = {B:.6f}")
    print(f"  C = {C:.6f}")
    print(f"  RMSE = {rmse(n_vals, n_fit):.3e}")

    # (Optional) Kurzreport der Residuen:
    print("\nResiduen (n_meas − n_fit):")
    for l, r in zip(lam_nm, resid):
        print(f"  λ = {l:7.2f} nm | Residuum = {r:+.3e}")
else:
    print("\nCauchy-Fit übersprungen (zu wenige Punkte oder deaktiviert).")




=== TEIL 1 — DISPERSION DER PRISMA ===

Berechnete n(λ):
  λ =  579.07 nm | δ_min = 41.10° | n = 1.544359
  λ =  576.96 nm | δ_min = 41.00° | n = 1.543249
  λ =  546.07 nm | δ_min = 40.80° | n = 1.541026
  λ =  491.60 nm | δ_min = 40.70° | n = 1.539913
  λ =  435.83 nm | δ_min = 40.50° | n = 1.537684

Cauchy-Fit (λ in μm):   n(λ) = A + B/λ^2 + C/λ^4
  A = 1.566383
  B = -0.010531
  C = 0.000969
  RMSE = 6.000e-04

Residuen (n_meas − n_fit):
  λ =  579.07 nm | Residuum = +7.655e-04
  λ =  576.96 nm | Residuum = -2.407e-04
  λ =  546.07 nm | Residuum = -9.355e-04
  λ =  491.60 nm | Residuum = +5.190e-04
  λ =  435.83 nm | Residuum = -1.083e-04


In [12]:
# =========================
# TEIL 2 — ABSORPTIONS-SPEKTROSKOPIE
# =========================

# ---------- EINGABEDATEN (nur Arrays) ----------
# Option A: synthetische Demo (True) — oder False und eigene Arrays füllen.
USE_SYNTHETIC_DEMO = True

# A) Synthese-Parameter (nur wenn Demo):
SYN_MIN_NM = 400.0
SYN_MAX_NM = 900.0
SYN_STEP   = 0.5

# B) Eigene Arrays (nur wenn USE_SYNTHETIC_DEMO=False):
wl_nm_user: List[float] = []
I0_user:    List[float] = []  # Referenz (ohne Probe)
Dark_user:  List[float] = []  # Dunkel/Offset
Id_user:    List[float] = []  # Mit Probe

# Reflexion berücksichtigen? T_corr = T / (1 − R)^2
REFLECTANCE_R: Optional[float] = None  # z. B. 0.04 oder None

# Peak-Suche (in OD)
PEAK_MIN_PROMINENCE = 0.02
PEAK_MIN_DISTANCE_NM = 2.0

# Referenzlinien für Zuordnung
ND_LINES_NM = [510, 522, 578, 740, 799, 868]
PR_LINES_NM = [444, 468, 481, 590]

# ---------- Synthese oder eigene Daten ----------
if USE_SYNTHETIC_DEMO:
    wl = np.arange(SYN_MIN_NM, SYN_MAX_NM + SYN_STEP, SYN_STEP)
    Dark = 20.0 + 2.0 * np.sin(wl / 50.0)
    I0 = 1000.0*np.exp(-((wl-650.0)/400.0)**2) + 300.0*np.exp(-((wl-450.0)/200.0)**2) + 50.0
    I0 = I0 + 0.02 * I0 * np.sin(wl / 25.0)
    # Linien (Nd/Pr) als Gauss-Absorber:
    alpha = np.zeros_like(wl)
    for line in ND_LINES_NM: alpha += 0.7*np.exp(-0.5*((wl-line)/1.2)**2)
    for line in PR_LINES_NM: alpha += 0.3*np.exp(-0.5*((wl-line)/1.2)**2)
    T_true = np.exp(-alpha * 0.02)
    Id = (I0 - Dark) * T_true + Dark
else:
    wl  = np.array(wl_nm_user, dtype=float)
    I0  = np.array(I0_user,    dtype=float)
    Dark= np.array(Dark_user,  dtype=float)
    Id  = np.array(Id_user,    dtype=float)
    assert len(wl)>0, "Eigene Arrays sind leer."
    assert len(wl)==len(I0)==len(Dark)==len(Id), "Alle Arrays müssen gleiche Länge haben."

# ---------- Transmission, Absorbanz, OD ----------
I0c = np.clip(I0 - Dark, 1e-12, None)
Idc = np.clip(Id - Dark, 1e-12, None)
T = safe_div(Idc, I0c)
T = np.clip(T, 1e-9, 1.0)

if REFLECTANCE_R is not None:
    T_corr = safe_div(T, (1.0 - REFLECTANCE_R)**2)
    T_corr = np.clip(T_corr, 1e-9, 1.0)
else:
    T_corr = T

A  = -np.log(T_corr)
OD = -np.log10(T_corr)

print("\n=== TEIL 2 — ABSORPTIONS-SPEKTROSKOPIE ===")

print("\nBeispielwerte (erste 10 Punkte):")
for i in range(min(10, len(wl))):
    print(f"  λ={wl[i]:.2f} nm | T={T_corr[i]:.4f} | A={A[i]:.4f} | OD={OD[i]:.4f}")

# ---------- Peak-Suche und Zuordnung ----------
idxs = find_peaks_simple(wl, OD, PEAK_MIN_PROMINENCE, PEAK_MIN_DISTANCE_NM)
peaks_nm  = [float(wl[i]) for i in idxs]
peaks_val = [float(OD[i]) for i in idxs]

print(f"\nGefundene Peaks in OD: {len(peaks_nm)}")
for lam, val in zip(peaks_nm, peaks_val):
    print(f"  λ≈{lam:.2f} nm | OD≈{val:.3f}")

nd_matches = nearest_lines(peaks_nm, ND_LINES_NM, tol_nm=2.5)
pr_matches = nearest_lines(peaks_nm, PR_LINES_NM, tol_nm=2.5)

def fmt_match(name: str, matches) -> str:
    if not matches: return f"{name}: (keine Peaks gefunden)"
    rows = []
    for q, r, d in matches:
        if r is None:
            rows.append(f"  gemessen {q:.2f} nm  →  kein Treffer (Δ={d:.2f} nm)")
        else:
            rows.append(f"  gemessen {q:.2f} nm  →  Referenz {r:.0f} nm (Δ={d:.2f} nm)")
    return name + ":\n" + "\n".join(rows)

print("\nZuordnung (±2.5 nm):")
print(fmt_match("Neodym", nd_matches))
print(fmt_match("Praseodym", pr_matches))

nd_count = sum(1 for _, r, _ in nd_matches if r is not None)
pr_count = sum(1 for _, r, _ in pr_matches if r is not None)

if nd_count > pr_count:
    verdict = "Wahrscheinlicher: Neodym (Nd)"
elif pr_count > nd_count:
    verdict = "Wahrscheinlicher: Praseodym (Pr)"
else:
    verdict = "Unentschieden / Mischprobe oder zu wenig Peaks"

print("\nFazit:", verdict)



=== TEIL 2 — ABSORPTIONS-SPEKTROSKOPIE ===

Beispielwerte (erste 10 Punkte):
  λ=400.00 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=400.50 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=401.00 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=401.50 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=402.00 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=402.50 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=403.00 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=403.50 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=404.00 nm | T=1.0000 | A=-0.0000 | OD=-0.0000
  λ=404.50 nm | T=1.0000 | A=-0.0000 | OD=-0.0000

Gefundene Peaks in OD: 0

Zuordnung (±2.5 nm):
Neodym: (keine Peaks gefunden)
Praseodym: (keine Peaks gefunden)

Fazit: Unentschieden / Mischprobe oder zu wenig Peaks
